In [ ]:
# Ref: https://api.python.langchain.com/en/latest/evaluation/langchain.evaluation.criteria.eval_chain.CriteriaEvalChain.html

In [7]:
import langchain_community
from langchain.evaluation.criteria import CriteriaEvalChain
from langchain.evaluation.criteria import LabeledCriteriaEvalChain

In [9]:
# What llms do we have?
! ollama list

NAME            	ID          	SIZE  	MODIFIED   
llama2:chat     	78e26419b446	3.8 GB	4 days ago	
tinyllama:latest	2644915ede35	637 MB	8 days ago	
llama3:8b       	365c0bd3c000	4.7 GB	8 days ago	
mistral:latest  	2ae6f6dd7a3d	4.1 GB	8 days ago	
phi3:latest     	64c1188f2485	2.4 GB	8 days ago	


In [23]:
# 1.0
from langchain_community.llms import Ollama
from langchain_community.chat_models.ollama import ChatOllama

# 1.0.1

llm= ChatOllama(model = "llama3:8b",    # This is also the default
             temperature=0.9,    # Default is None (ie 0.8)
             num_predict=256      # Maximum number of tokens to predict when generating text
                                 #  (Default: 128, -1 = infinite generation, -2 = fill context)
           )


# llm = Ollama()
llm

ChatOllama(model='llama3:8b', num_predict=256, temperature=0.9)

In [24]:
criteria = {"my-custom-criterion": "Is the submission the most amazing ever?"}
evaluator = CriteriaEvalChain.from_llm(llm=llm, criteria=criteria)
evaluator.evaluate_strings(prediction="Imagine an ice cream flavor for the color aquamarine", input="Tell me an idea")


{'reasoning': 'Here\'s my step-by-step reasoning:\n\n1. Is the submission an idea?\n\t* Yes, the submission is "Imagine an ice cream flavor for the color aquamarine", which is an idea.\n2. Does the submission relate to the input ("Tell me an idea")?\n\t* Yes, the submission is a response to the request to provide an idea.\n3. Does the submission meet my-custom-criterion ("Is the submission the most amazing ever")?\n\t* This criterion is subjective and hard to evaluate objectively. However, I\'ll assume that it\'s meant to be evaluated based on creativity, originality, and potential impact.\n\t* Upon reviewing the submission, I think that "Imagine an ice cream flavor for the color aquamarine" is a creative and unique idea that might spark interesting conversations about colors and flavors. While it may not be the most amazing ever, it\'s certainly a thoughtful and imaginative response.\n\nBased on my evaluation, here\'s my answer:\n\nY',
 'value': 'Y',
 'score': 1}

In [13]:
# 1.0.1

llm= Ollama(model = "mistral",    # This is also the default
             temperature=0.9,    # Default is None (ie 0.8)
             num_predict=256      # Maximum number of tokens to predict when generating text
                                 #  (Default: 128, -1 = infinite generation, -2 = fill context)
           )


# llm = Ollama()
llm

Ollama(model='mistral', num_predict=256, temperature=0.9)

In [8]:
criteria = "correctness"
evaluator = LabeledCriteriaEvalChain.from_llm(
    llm=llm,
    criteria=criteria,
)
evaluator.evaluate_strings(
  prediction="The answer is 4",
  input="How many apples are there?",
  reference="There are 3 apples",
  )

{'reasoning': "To determine if the submission meets the Criteria, let's break down the analysis:\n\n1. Correctness: A correct answer is one that matches with the provided reference in terms of the number of apples.\n2. Reference: There are 3 apples according to the given data.\n3. Submission: The submission states there are 4 apples.\n\nComparing the Submission and the Reference, we find that they do not match - the answer in the reference is 3 while the submission claims it's 4. Therefore, based on our analysis, the answer is incorrect.\n\nY\n(Repeat the letter for clarity)",
 'value': 'Y',
 'score': 1}

In [ ]:
#from langchain.evaluation import load_evaluator
#from langchain import open_ai
from langchain.evaluation.loading import load_evaluator

evaluator = load_evaluator("criteria", llm=llm, criteria="conciseness")



In [26]:
from langchain.evaluation import load_evaluator
from langchain.chat_models import ChatOllama
from langchain.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings

In [37]:
#This is work
evaluator = load_evaluator("labeled_score_string", llm=ChatOllama(model="llama2"))
evaluator = load_evaluator("pairwise_string",  llm=Ollama(model="llama3:8b"))

This chain was only tested with GPT-4. Performance may be significantly worse with other models.


In [38]:
# https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/comparison/pairwise_string/
# https://stackoverflow.com/q/78231114/3282777

evaluator.evaluate_string_pairs(
    prediction="there are three dogs",
    prediction_b="4",
    input="how many dogs are in the park?",
    reference="four",
)

/home/ashok/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain/evaluation/schema.py:128: UserWarning: Ignoring reference in PairwiseStringEvalChain, as it is not expected.
To use a reference, use the LabeledPairwiseStringEvalChain (EvaluatorType.LABELED_PAIRWISE_STRING) instead.
  warn(self._skip_reference_warning)


{'reasoning': 'I will now compare and evaluate the responses provided by the two AI assistants to the user question "how many dogs are in the park?".\n\nFrom my analysis, I can see that both responses aim to directly answer the user\'s question. However, there is a significant difference between the quality of their answers.\n\nAssistant A provides a response that is brief and simplistic, stating that "there are three dogs". While this response attempts to answer the user\'s question, it lacks depth, context, and relevance. It does not provide any additional information or insights that might be helpful to the user. The response also relies heavily on guesswork, as there is no evidence provided to support the claim.\n\nOn the other hand, Assistant B provides a direct and accurate response stating "4". This answer is straightforward, concise, and relevant to the question. It does not attempt to provide additional information or context, but it accurately answers the user\'s query.\n\nIn

In [28]:
# This is equivalent to loading using the enum
from langchain.evaluation import EvaluatorType

evaluator = load_evaluator(EvaluatorType.CRITERIA, criteria="conciseness")

ValueError: Evaluation with the <class 'langchain.evaluation.criteria.eval_chain.CriteriaEvalChain'> requires a language model to function. Failed to create the default 'gpt-4' model. Please manually provide an evaluation LLM or check your openai credentials.